<a href="https://colab.research.google.com/github/gencross/NewtonGroup1/blob/main/Main_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Newtons Method can be used to find the roots of a differentiable function $f(x)$. Given a starting guess, $x_{0}$, $x_{n+1}$ is defined as $x_{n} - \frac{f(x_{n})}{f'(x_{n})} $. Depending on the choice of $x_{0}$, the series $x_{n}$ may approach different real and complex roots of the function. 

In [ ]:
print("Hello, world!")

Hello, world!
